# Assignment: Amazon Sales Data

# Task 1: Create a new Jupyter Notebook, create a virtual environment. Load the required dataset and display first few lines of the data.

Github repo link: https://github.com/sayashhad/Data-Analyst-Internship-Assignment

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
#Task 1: Loading the dataset and displaying it.
df = pd.read_csv("assets/Amazon_Sale_Report.csv")
df.head()

/var/folders/my/hsss9j1j58zgjjlg548zbmxw0000gp/T/ipykernel_27767/1241508460.py:2: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("assets/Amazon_Sale_Report.csv")


,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN


# Task 2: Clean the dataset by identifying the inconsistent, wrong and missing values.

#### Checking for datatypes to check if there are any inconsistencies with the data

In [3]:
df['Status'].unique()

array(['Cancelled', 'Shipped - Delivered to Buyer', 'Shipped',
       'Shipped - Returned to Seller', 'Shipped - Rejected by Buyer',
       'Shipped - Lost in Transit', 'Shipped - Out for Delivery',
       'Shipped - Returning to Seller', 'Shipped - Picked Up', 'Pending',
       'Pending - Waiting for Pick Up', 'Shipped - Damaged', 'Shipping'],
      dtype=object)

In [4]:
print(df.dtypes)

index                   int64
Order ID               object
Date                   object
Status                 object
Fulfilment             object
Sales Channel          object
ship-service-level     object
Style                  object
SKU                    object
Category               object
Size                   object
ASIN                   object
Courier Status         object
Qty                     int64
currency               object
Amount                float64
ship-city              object
ship-state             object
ship-postal-code      float64
ship-country           object
promotion-ids          object
B2B                      bool
fulfilled-by           object
Unnamed: 22            object
dtype: object


#### Key takeaways: Date is an object, whereas it should be in datetime format. There exists a column that is unnamed and does not provide any information.

#### 1. Change the date type to datetime format.

In [5]:
#Convert date from string type to datetime type
df['Date'] = pd.to_datetime(df['Date'], format = '%m-%d-%y');

#### Checking for values that need to be standardized. In this case, there are more than required states. Hence needs standardization to required number of states:

In [6]:
# for cols in df.columns:
#     print(cols)
#     print(df[cols].unique())
#     print('--' *40)
for col in df.columns:
    print(f"{col}: {df[col].nunique()}")

index: 128975
Order ID: 120378
Date: 91
Status: 13
Fulfilment: 2
Sales Channel : 2
ship-service-level: 2
Style: 1377
SKU: 7195
Category: 9
Size: 11
ASIN: 7190
Courier Status: 3
Qty: 10
currency: 1
Amount: 1410
ship-city: 8955
ship-state: 69
ship-postal-code: 9459
ship-country: 1
promotion-ids: 5787
B2B: 2
fulfilled-by: 1
Unnamed: 22: 1


### Checking for the null values present and then arranging them in descending order to check which columns require more attention.

In [7]:
(df.isna().mean()*100).sort_values(ascending = False)

fulfilled-by          69.546811
promotion-ids         38.110487
Unnamed: 22           38.030626
currency               6.043807
Amount                 6.043807
Courier Status         5.328164
ship-country           0.025586
ship-postal-code       0.025586
ship-state             0.025586
ship-city              0.025586
ship-service-level     0.000000
Style                  0.000000
Date                   0.000000
B2B                    0.000000
Status                 0.000000
Fulfilment             0.000000
Sales Channel          0.000000
Qty                    0.000000
Order ID               0.000000
ASIN                   0.000000
Size                   0.000000
Category               0.000000
SKU                    0.000000
index                  0.000000
dtype: float64

#### Understanding the relationship between columns to get the missing values

In [8]:
df[df['Courier Status'].isna()]['Status'].unique()
# df['Courier Status'].unique()

array(['Cancelled', 'Shipped - Delivered to Buyer',
       'Shipped - Returned to Seller'], dtype=object)

#### Takeaway: For every missing values for Courier-Status, there exists a value for Status based on which the values can be filled to either 'Cancelled or 'Shipped' based on the Status.

### 2. Add the missing values accordingly

In [9]:
# If the Status of the row is Cancelled, change the missing values to 'Cancelled as well'
df.loc[(df['Courier Status'].isna()) & (df['Status'] == 'Cancelled'), 'Courier Status'] = 'Cancelled'

# For the rest of them, change it to 'Shipped'
df['Courier Status'] = df['Courier Status'].fillna('Shipped')

### 3. Checking if there exists orders that have inconsistent quantities, like float or null. Since there exists rows with 0 quantity, these records are erroranous and must be removed

In [10]:
# df['Qty'].unique()

# to_drop = df[(df['Qty'] == 0) & (df['Amount'] > 0)].index

# df.drop(index = to_drop, inplace = True)

In [11]:
# Checking for the relation between status of order and for the rows that have missing values in amount column.
# df[df['Amount'].isna()]['Qty'].unique()
df[df['Style']=='JNE3797']['Amount'].value_counts()

Amount
735.00     1546
771.00     1369
725.00      455
700.00      142
734.29      115
715.00       97
724.00       73
0.00         70
799.00       37
690.48       31
761.00       18
680.95       13
734.28        6
689.52        5
724.76        3
1542.00       3
1470.00       3
1450.00       2
760.95        2
1430.00       1
2860.00       1
680.96        1
2175.00       1
1598.00       1
Name: count, dtype: int64

### 4. Standardizing the values for categorical data like fulfilment, Sales Channel, Ship service level, State names etc.

In [12]:
df['Fulfilment'] = df['Fulfilment'].str.lower()
df['Sales Channel '] = df['Sales Channel '].str.lower()
df['ship-service-level'] = df['ship-service-level'].str.lower()
df['Courier Status'] = df['Courier Status'].str.lower()


df['ship-state'] = df['ship-state'].str.strip().str.title()
state_mapping = {
    'New Delhi': 'Delhi',
    'Rajshthan': 'Rajasthan',
    'Rajsthan': 'Rajasthan',
    'Rj': 'Rajasthan',
    'Pb': 'Punjab',
    'Punjab/Mohali/Zirakpur': 'Punjab',
    'Orissa': 'Odisha',
    'Nl': 'Nagaland',
    'Ar': 'Arunachal Pradesh',
    'Arunachal Pradesh': 'Arunachal Pradesh',
    'Pondicherry': 'Puducherry',
    'Dadra And Nagar': 'Dadra and Nagar Haveli',
    'Ap': 'Andhra Pradesh',
    'Apo': 'Armed Forces Post Office',
}

df['ship-state'] = df['ship-state'].replace(state_mapping)
# df['ship-state'].unique()

#### 5. Also seggregating data of address data to make a heatmap with location

In [13]:
address_df = df[~df['ship-city'].isna()][['ship-city', 'ship-state','ship-country','ship-postal-code']]

### 6. Removing columns that have a single value, or does not contribute to the data set, or have multiple missing values.

In [14]:
fulfilled_df = df['fulfilled-by']
promotion_df = df['promotion-ids']
df = df.drop(columns = ['promotion-ids', 'currency', 'ship-country', 'fulfilled-by', 'Unnamed: 22'])
df.head()

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,Amount,ship-city,ship-state,ship-postal-code,B2B
0,0,405-8078784-5731545,2022-04-30,Cancelled,merchant,amazon.in,standard,SET389,SET389-KR-NP-S,Set,S,B09KXVBD7Z,cancelled,0,647.62,MUMBAI,Maharashtra,400081.0,False
1,1,171-9198151-1101146,2022-04-30,Shipped - Delivered to Buyer,merchant,amazon.in,standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,B09K3WFS32,shipped,1,406.00,BENGALURU,Karnataka,560085.0,False
2,2,404-0687676-7273146,2022-04-30,Shipped,amazon,amazon.in,expedited,JNE3371,JNE3371-KR-XL,kurta,XL,B07WV4JV4D,shipped,1,329.00,NAVI MUMBAI,Maharashtra,410210.0,True
3,3,403-9615377-8133951,2022-04-30,Cancelled,merchant,amazon.in,standard,J0341,J0341-DR-L,Western Dress,L,B099NRCT7B,cancelled,0,753.33,PUDUCHERRY,Puducherry,605008.0,False
4,4,407-1069790-7240320,2022-04-30,Shipped,amazon,amazon.in,expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,B098714BZP,shipped,1,574.00,CHENNAI,Tamil Nadu,600073.0,False


#### Trying to find if there exists a relation between amount and any other attributes to fill the missing values

#### Couldn't find any particular correlation between missing values of amount and any other column. 

In [15]:
(df.isna().mean()*100).sort_values(ascending = False)

Amount                6.043807
ship-postal-code      0.025586
ship-state            0.025586
ship-city             0.025586
index                 0.000000
Size                  0.000000
Qty                   0.000000
Courier Status        0.000000
ASIN                  0.000000
Category              0.000000
Order ID              0.000000
SKU                   0.000000
Style                 0.000000
ship-service-level    0.000000
Sales Channel         0.000000
Fulfilment            0.000000
Status                0.000000
Date                  0.000000
B2B                   0.000000
dtype: float64

### 7. Creating New columns: Day, Successful orders, Total items

In [25]:
#Tells whelther the order is successful yet or not
successful_list = ['Shipped', 'Shipped - Delivered to Buyer']
df['Order Sucess'] = df['Status'].isin(successful_list)

#Tells the weekday of the order based on the date
df['Weekday'] = df['Date'].dt.day_name()

#Denotes the total items(not Qty per item) ordered by the user
df['Total items'] = df.groupby('Order ID')['Order ID'].transform('count')

### 8. Saving different data as multiple CSV files

In [26]:
df.to_csv("NewDataset/cleaned_dataset.csv", index = False)
address_df.to_csv("NewDataset/cleaned_addressdata.csv", index = True)